In [5]:
# .log 데이터를 예측하는 과정입니다.
import joblib
import datetime
import pandas as pd
model = joblib.load('../hardvoting.pkl')
columns = ["ts", "uid", "id.orig_h", "id.orig_p", "id.resp_h", "id.resp_p", "proto", 
          "service", "duration", "orig_bytes", "resp_bytes", "conn_state", "local_orig", 
          "local_resp", "missed_bytes", "history", "orig_pkts", "orig_ip_bytes", "resp_pkts", 
          "resp_ip_bytes", "tunnel_parents", "ip_proto"]

# 데이터를 가장 먼저 불러오고요!
df = pd.read_csv("conn.log", delimiter="\t", comment="#", skip_blank_lines=True)
df.columns = columns

# 기존 데이터셋을 바탕으로 특성 공학을 적용시킵니다. '-'이 값에 대해서는 0으로 변경해주도록 하겠습니다.
# 0으로 변경하면 해당 특성은 영향을 받지 않습니다.
df['orig_bytes'] = df['orig_bytes'].replace('-', 0).astype(int)
df['orig_ip_bytes'] = df['orig_ip_bytes'].replace('-', 0).astype(int)
df['src_bytes'] = df['orig_bytes'].astype(int)+df['orig_ip_bytes'].astype(int)
df['service'] = df['service'].replace('-', 'unknown').astype(object)
df['protocol_type'] = df['proto'].replace('-', 'unknown').astype(object)
df['same_srv_rate'] = df.groupby('id.orig_h')['service'].transform(lambda x: x.eq(x.iloc[-1]).mean()).replace('-', 0).astype(int)
df['rerror_rate'] = df['conn_state'].apply(lambda x: 1 if x in ['REJ', 'S0'] else 0).replace('-', 0).astype(int)
df['srv_rerror_rate'] = df.groupby('service')['conn_state'].transform(lambda x: x.isin(['REJ', 'S0']).mean()).replace('-', 0).astype(int)
df['dst_host_srv_count'] = df.groupby('id.resp_h')['id.resp_h'].transform('count').replace('-', 0).astype(int)
df['dst_host_same_src_port_rate'] = df.groupby('id.resp_h')['id.orig_p'].transform(lambda x: x.eq(x.iloc[-1]).mean()).replace('-', 0).astype(int)
df['wrong_fragment'] = df['history'].apply(lambda x: 1 if 'f' in str(x) or 'r' in str(x) else 0).replace('-', 0).astype(int)
df['dst_host_rerror_rate'] = df.groupby('id.resp_h')['conn_state'].transform(lambda x: x.isin(['REJ', 'S0']).mean()).replace('-', 0).astype(int)

# 특성 공학을 통해 새로운 특성을 만들었으니, 이제는 불필요한 특성을 제거하도록 하겠습니다.
detected = [i for i in df.columns if i in model.feature_names_in_]
remake_df = df[detected]
remake_df = remake_df.reindex(columns=model.feature_names_in_, fill_value=0)
result = model.predict(remake_df)

# 공격에 대한 매핑을 진행을 해줍시다.
mapping_attack = {0: 'DOS_Attack', 1: 'NORMAL', 2: 'PROBING_Attack', 3: 'R2L_Attack', 4: 'U2R_Attack'}
ip_addr = df['id.orig_h'].values
track_time = df['ts'].apply(lambda x: datetime.datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))

# 결과를 출력합니다.
for i in range(len(result)):
    if result[i]==1:
        continue
    else:
        print('-'*80)
        print(f'{track_time[i]} -> {ip_addr[i]}은 현재 {mapping_attack[result[i]]}이 의심스럽습니다.')

--------------------------------------------------------------------------------
2025-03-20 12:52:09 -> 192.168.45.231은 현재 R2L_Attack이 의심스럽습니다.
--------------------------------------------------------------------------------
2025-03-20 12:52:09 -> fe80::186c:b5c4:28a4:291f은 현재 R2L_Attack이 의심스럽습니다.
--------------------------------------------------------------------------------
2025-03-20 12:52:28 -> 192.168.45.184은 현재 R2L_Attack이 의심스럽습니다.
--------------------------------------------------------------------------------
2025-03-20 12:52:28 -> fe80::c6b:5da0:725f:1870은 현재 R2L_Attack이 의심스럽습니다.
--------------------------------------------------------------------------------
2025-03-20 12:52:07 -> fe80::623e:ba80:5764:dd33은 현재 PROBING_Attack이 의심스럽습니다.
--------------------------------------------------------------------------------
2025-03-20 12:52:37 -> 192.168.45.211은 현재 PROBING_Attack이 의심스럽습니다.
--------------------------------------------------------------------------------
2025-03-20 12:5

/Users/hack/PycharmProjects/threatlensV2/.venv/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
